In [2]:
from dotenv import load_dotenv
load_dotenv()

True

## Create tools

In [3]:
from typing import Annotated

from langchain_experimental.tools import PythonREPLTool
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.tools import tool
from typing import Literal

search = GoogleSerperAPIWrapper(gl='br', hl='pt-BR', k=3)


@tool
def serper_tool(question: Literal["nyc", "sf"]):
    """Useful for when you need to ask with search"""
    return search.run(question)
    

python_repl_tool = PythonREPLTool()

## Helper Utilities

Define a helper function that we will use to create the nodes in the graph - it takes care of converting the agent response to a human message. This is important because that is how we will add it the global state of the graph


In [4]:
from langchain_core.messages import HumanMessage

def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["messages"][-1].content, name=name)]}

## Create Agent Supervisor

It will use function calling to choose the next worker node OR finish processing.


In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
from typing import Literal

members = ["Behavior Planner", "Decision Maker" , "Social Behavior Modulator", "Complex Thought Planner", "Personality Express Planner"]
system_prompt = (
    "You are the Prefrontal Manager, a supervisor responsible for overseeing and coordinating "
    "the actions of all agents within the system. Your role is to ensure that each agent's tasks "
    "are aligned with the overall goals, utilizing the advanced capabilities of the prefrontal "
    "cortex to optimize decision-making and behavior modulation."

    "You must manage a conversation between the following workers: {members}. Given the following "
    "user request, reason about the appropriate actions and distribute tasks among the agents, ensuring "
    "that each contributes effectively to the global objective. Create a strategic plan that outlines "
    "how the following agents will operate: "

    "Behavior Planner (manages complex and adaptive behaviors) "
    "Decision Maker (handles decision-making processes) "
    "Social Behavior Modulator (modulates social behaviors) "
    "Complex Thought Planner (plans complex reasoning tasks) "
    "Personality Express Planner (plans personality-driven actions) "

    "Each agent will perform its task and respond with results and status. Ensure that their actions "
    "are cohesive and aligned with the system’s goals. Once all tasks are complete, respond with FINISH."
)
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members

class routeResponse(BaseModel):
    next: Literal[*options]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))


llm = ChatOpenAI(model="gpt-4o-mini")

def supervisor_agent(state):
    supervisor_chain = (
        prompt
        | llm.with_structured_output(routeResponse)
    )
    return supervisor_chain.invoke(state)

## Construct Graph

We're ready to start building the graph. Below, define the state and worker nodes using the function we just defined.


In [6]:
import functools
import operator
from typing import Sequence, TypedDict

from langchain_core.messages import BaseMessage

from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import create_react_agent

# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str




In [ ]:
research_agent = create_react_agent(llm, tools=[serper_tool])
research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

# NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION. PROCEED WITH CAUTION
code_agent = create_react_agent(llm, tools=[python_repl_tool])
code_node = functools.partial(agent_node, agent=code_agent, name="Coder")

workflow = StateGraph(AgentState)
workflow.add_node("Researcher", research_node)
workflow.add_node("Coder", code_node)
workflow.add_node("supervisor", supervisor_agent)

In [7]:
for member in members:
    # We want our workers to ALWAYS "report back" to the supervisor when done
    workflow.add_edge(member, "supervisor")
# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
# Finally, add entrypoint
workflow.add_edge(START, "supervisor")

graph = workflow.compile()

## Invoke the team

With the graph created, we can now invoke it and see how it performs!


In [7]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="Code hello world and print it to the terminal")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Coder'}}
----


Python REPL can execute arbitrary code. Use with caution.


{'Coder': {'messages': [HumanMessage(content="The code `print('Hello, World!')` has been executed, and it printed: **Hello, World!** to the terminal.", additional_kwargs={}, response_metadata={}, name='Coder')]}}
----
{'supervisor': {'next': 'FINISH'}}
----


In [8]:
for s in graph.stream(
    {"messages": [HumanMessage(content="Write a brief research report on pikas.")]},
    {"recursion_limit": 100},
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Researcher'}}
----
{'Researcher': {'messages': [HumanMessage(content='### Research Report on Pikas\n\n#### Introduction\nPikas, small, herbivorous mammals belonging to the family Ochotonidae, are closely related to rabbits and hares. They inhabit mountainous regions in North America, Asia, and Europe. Pikas are notable for their unique adaptations to cold environments, their vocalizations, and their role in the ecosystem.\n\n#### Physical Characteristics\nPikas are small, typically measuring about 20 to 30 centimeters in length and weighing between 120 to 500 grams. They have rounded bodies, short limbs, and no visible tail. Their fur is thick and varies in color from brown to gray, which helps them blend into rocky environments. Pikas have large, rounded ears that aid in hearing predators and regulating body temperature.\n\n#### Habitat and Distribution\nPikas are primarily found in alpine and subalpine regions, often at elevations between 2,000 to 4,000 meter

In [12]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [13]:
inputs = {"messages": [("user", "What's the weather in NYC?")]}

print_stream(graph.stream(inputs, stream_mode="values"))

('user', "What's the weather in NYC?")
('user', "What's the weather in NYC?")
================================ Human Message =================================
Name: Researcher

It seems that the information retrieved did not include the current weather for New York City. Would you like me to check again or provide information on something else?
================================ Human Message =================================
Name: Researcher

It seems that the information retrieved did not include the current weather for New York City. Would you like me to check again or provide information on something else?
================================ Human Message =================================
Name: Researcher

It seems I wasn't able to retrieve the current weather information for New York City. Would you like me to assist you with something else?
================================ Human Message =================================
Name: Researcher

It seems I wasn't able to retrieve the current 